# Lab 1 - Análise Preditiva com Rede Neural

Este script 
demonstra um exemplo prático de utilização de análise preditiva com um modelo de redes neurais. O exercício faz parte da formação Inteligência Artificial da Data Science Academy. 

O modelo treinado passa por todas as etapas principais para preparação, treino e avaliação do modelo de rede neural. Neste exerício foi usando o Pytorch e o Ligthning com o uso de linguagem Python.

## 1. Instalando e carregando pacotes

In [1]:
# Versão do python
from platform import python_version
print('Versão da linguagem Python usada neste Jupyter Notebook é: ', python_version())

Versão da linguagem Python usada neste Jupyter Notebook é:  3.9.13


In [2]:
# instala o watermark
#!pip install -q -U watermark

In [3]:
# instala o pytorch e lightning
#!pip install -q torch==1.13.0

In [4]:
# instala o pytorch e lightning
#!pip install -q pytorch-lightning==1.8.3

In [5]:
# Carga dos pacotes

# Pacotes para check de hardware e versões
import gc
import types
import pkg_resources

# Pytorch e Pytorch Lightning
import torch
import pytorch_lightning as pl

# Pacote de rede neural e otimizador
from torch import nn, optim

# Pacote para prepara o dataset de exemplo
from torch.autograd import Variable

# Pacote para preparar os dados para treinar o modelo
from torch.utils.data import DataLoader

# Pacote para gravar o modelo em disco
from pytorch_lightning.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')

In [6]:
# Versões dos pacotes utilizados neste Jupyter Notebook
%reload_ext watermark
%watermark -a "Lab1 - IA" --iversions

Author: Lab1 - IA

pytorch_lightning: 1.8.3
torch            : 1.13.0



## 2. Verificando o ambiente de desenvolvimento

In [7]:
# Verificando o dispositivo
processing_device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
# verifica resultado
print(processing_device)

cpu


In [9]:
# Verifica se a GPU pode ser utilizado
# Nesse caso o resultado esperado é FALSE
torch_aval = torch.cuda.is_available()

In [10]:
# Verifica resultado
print(torch_aval)

False


In [11]:
# Labels para o relatório de verificação
label_1 = 'Visão Geral do Ambiente'
label_2 = 'Se NVIDIA não for encontrado, então CUDA não está disponível'
label_3 = 'Fim da Verificação'

In [12]:
# Função que verifica o que está importado nesta seção
def get_imports():
    
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split('.')[0]
            
        elif isinstance(val, type):
            name = val.__module__.split('.')[0]
        
        poorly_named_packages = {'PIL': 'Pillow', 'sklearn': 'scikit-learn'}
        
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
        
        yield name

In [13]:
# Pacotes importados neste jupyter notebook
imports = list(set(get_imports()))
print(imports)

['pytorch_lightning', '_i13', 'In', '_iii', '_i9', '__', '_i8', 'warnings', 'builtins', '_i6', 'gc', '_i11', '_i12', 'get_imports', '__name__', '_oh', 'types', 'torch_aval', '_i', '__spec__', '_i1', 'label_2', '_dh', '_i3', '_i10', 'torch', 'exit', '_i2', 'quit', 'label_3', '__package__', 'python_version', 'Out', 'processing_device', '_i4', '_ih', '___', '_i5', 'pkg_resources', 'label_1', '_i7', 'get_ipython', '_', '__doc__', '__loader__', '_ii']


In [14]:
# Loop para verificar os requerimentos de instalação dos pacotes
requeriments = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!='pip':
        requeriments.append((m.project_name, m.version))

In [15]:
# Verifica o resultado
print(requeriments)

[('torch', '1.13.0')]


In [16]:
# Cria pasta de dados
pasta_dados = r'dados'

In [17]:
# Print do resumo 
print(f'{label_1:-^100}')
print()
print(f'Pasta de Dados: ', pasta_dados)
print(f'Versões dos Pacotes Requeridos: ', requeriments)
print(f'Dispositivo que será usado para treinar o modelo: ', processing_device)
print(f'CUDA está disponível? ', torch_aval)
print('Versão do Pytorch: ', torch.__version__)
print('Versão do Lightning: ', pl.__version__)
print()
print(f'{label_2:-^100}')
!nvidia-smi
gc.collect()
print()
print(f'Limpando a memória da GPU (se disponível): ', torch.cuda.empty_cache())
print(f'{label_3:-^100}')

--------------------------------------Visão Geral do Ambiente---------------------------------------

Pasta de Dados:  dados
Versões dos Pacotes Requeridos:  [('torch', '1.13.0')]
Dispositivo que será usado para treinar o modelo:  cpu
CUDA está disponível?  False
Versão do Pytorch:  1.13.0+cpu
Versão do Lightning:  1.8.3

--------------------Se NVIDIA não for encontrado, então CUDA não está disponível--------------------

Limpando a memória da GPU (se disponível):  None
-----------------------------------------Fim da Verificação-----------------------------------------


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


## 3. Carga dos Dados

In [18]:
# Carga dos dados de entrada
dados_entrada = [Variable(torch.Tensor([0, 0])),
                 Variable(torch.Tensor([0, 1])),
                 Variable(torch.Tensor([1, 0])),
                 Variable(torch.Tensor([1, 1]))]

In [19]:
# Verifica os dados de entrada
type(dados_entrada)

list

In [20]:
# Carga dos dados de saída
dados_saida = [Variable(torch.Tensor([0])),
                 Variable(torch.Tensor([1])),
                 Variable(torch.Tensor([1])),
                 Variable(torch.Tensor([0]))]

In [21]:
# Verifica os dados de saída
type(dados_saida)

list

In [22]:
# Dataset final
dados_final = list(zip(dados_entrada, dados_saida))
dados_final

[(tensor([0., 0.]), tensor([0.])),
 (tensor([0., 1.]), tensor([1.])),
 (tensor([1., 0.]), tensor([1.])),
 (tensor([1., 1.]), tensor([0.]))]

In [23]:
# Verifica os dados final
type(dados_final)

list

In [24]:
# Prepara o data loader
loader_treinamento = DataLoader(dados_final, batch_size = 1)

In [25]:
# Verifica o tipo
type(loader_treinamento)

torch.utils.data.dataloader.DataLoader

In [26]:
class SimpleNet(pl.LightningModule):
  
    # Método construtor
    # Usado para inicializar os parâmetros do modelo
    def __init__(self):

        super(SimpleNet, self).__init__()  
        
        self.input_layer = nn.Linear(2, 4)
        self.output_layer = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()
        self.loss = nn.MSELoss()

    # Método da passada para frente (forward)
    def forward(self, input):
        x = self.input_layer(input)
        x = self.sigmoid(x)
        output = self.output_layer(x)
        return output

    # Método de otimização
    def configure_optimizers(self):
        params = self.parameters()
        optimizer = optim.Adam(params = params, lr = 0.01)
        return optimizer

    # Método das passadas de treinamento
    def training_step(self, batch, batch_idx):
        entrada, saida = batch
        outputs = self(entrada) 
        loss = self.loss(outputs, saida)
        return loss 

#### Mais informações no capítulo 15 do Deep Learning Book

## 4. Treino do Modelo com Rede Neural

In [27]:
# Cria o modelo
modelo = SimpleNet()

In [28]:
# Cria o callback de checkpoint
checkpoint_callback = ModelCheckpoint()

In [29]:
# Cria o trainer
trainer = pl.Trainer(devices = 1,
                     accelerator = 'cpu',
                     max_epochs = 100,
                     callbacks = [checkpoint_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [30]:
# Treinamento
trainer.fit(modelo, train_dataloaders = loader_treinamento)

Missing logger folder: C:\Users\morai\DSA_IA\DeepLearningPytorch\Cap02\Lab1\lightning_logs

  | Name         | Type    | Params
-----------------------------------------
0 | input_layer  | Linear  | 12    
1 | output_layer | Linear  | 5     
2 | sigmoid      | Sigmoid | 0     
3 | loss         | MSELoss | 0     
-----------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params
0.000     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [31]:
# Verifica tipo do device
modelo.device

device(type='cpu')

In [32]:
# Verifica a arquiterura do modelo
modelo.eval()

SimpleNet(
  (input_layer): Linear(in_features=2, out_features=4, bias=True)
  (output_layer): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (loss): MSELoss()
)

In [33]:
# Resumo do modelo
modelo.state_dict()

OrderedDict([('input_layer.weight',
              tensor([[ 0.4819,  0.0095],
                      [ 0.4522,  0.1973],
                      [-1.0225, -0.9093],
                      [-0.4804, -0.2510]])),
             ('input_layer.bias',
              tensor([-0.4570,  0.6250, -0.2941,  0.4637])),
             ('output_layer.weight',
              tensor([[-0.3625,  0.5242, -0.2409,  0.1075]])),
             ('output_layer.bias', tensor([0.2890]))])

## 5. Avaliação do Modelo

In [34]:
# Extrai o melhor modelo do checkpoint
melhor_modelo = (checkpoint_callback.best_model_path)
print(melhor_modelo)

C:\Users\morai\DSA_IA\DeepLearningPytorch\Cap02\Lab1\lightning_logs\version_0\checkpoints\epoch=99-step=400.ckpt


In [35]:
# Carrega o modelo do último checkpoint
modelo_final = modelo.load_from_checkpoint(checkpoint_callback.best_model_path)

In [36]:
# Verifica o resumo do modelo final
modelo_final.state_dict()

OrderedDict([('input_layer.weight',
              tensor([[ 0.4819,  0.0095],
                      [ 0.4522,  0.1973],
                      [-1.0225, -0.9093],
                      [-0.4804, -0.2510]])),
             ('input_layer.bias',
              tensor([-0.4570,  0.6250, -0.2941,  0.4637])),
             ('output_layer.weight',
              tensor([[-0.3625,  0.5242, -0.2409,  0.1075]])),
             ('output_layer.bias', tensor([0.2890]))])

In [37]:
# Novo registro de entrada
novo_dado_entrada_A = Variable(torch.Tensor([1,1]))

In [38]:
# Faz a previsão
pred_A = modelo_final(novo_dado_entrada_A)
print(pred_A)

tensor([0.5374], grad_fn=<AddBackward0>)


In [39]:
print('A previsão de classe de pred_A é: ', int(pred_A.round()))

A previsão de classe de pred_A é:  1


In [40]:
# Novos dados de entrada
novos_dados = [Variable(torch.Tensor([1,1])),
               Variable(torch.Tensor([0,0])),
               Variable(torch.Tensor([1,0])),
               Variable(torch.Tensor([0,1]))]

In [42]:
# Loop de test
for val in novos_dados:
    
    # Faz a previsão de cada registro
    previsao = modelo_final(val)
    
    # Imprime o resultado
    print([int(val [0]), int(val [1])], int(previsao.round()))

[1, 1] 1
[0, 0] 0
[1, 0] 0
[0, 1] 1


In [43]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "LAB 1 - DSA I.A" --iversions

Author: LAB 1 - DSA I.A

pytorch_lightning: 1.8.3
torch            : 1.13.0

